In [82]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import os.path

In [83]:
xl = pd.ExcelFile("data.xlsx")
sheets = xl.sheet_names
print(sheets)
new_columns = ['Name', 'List_ID', 'Year', 'Reformatted_Name',
       'Gender', 'is_from_London', 'London_Street',
       'London_Region', 'London_Lat_Long', 'City',
       'Non_London_City_Lat_Long', 'Nation', 'Other1', 'Other2']

geolocator = Nominatim()

['Emma Lyon 1812', 'Bolaffey 1820', 'Polack 1830', ' Moss 1839', ' Infant 1841', ' VOJ 1842-45', 'Belisario 1856', 'Project Details']


In [84]:
def get_location(street, region, city):
    try:
        location = geolocator.geocode(street+' '+region+' '+city, timeout=5)
        if location == None:
            location = geolocator.geocode(street+' '+city, timeout=5)
            if location == None:
                location = geolocator.geocode(region+' '+city, timeout=5)
                if location == None:
                    location = geolocator.geocode(city, timeout=5)
        return location
    except Exception:
        print(Exception)
        return None

In [85]:
def update_coordinates(df):
    new_df = df[df.ix[:,6:8]!=0].ix[:,[6,7]].dropna(axis=0, how='all', subset=['London_Street','London_Region'])
    print('Fetching locations................')
    for idx in new_df.index:
        street = ''
        region = ''
        city = ''
        if not pd.isnull(new_df.loc[idx,'London_Street']):
            street = new_df.loc[idx,'London_Street']
        if not pd.isnull(new_df.loc[idx,'London_Region']):
            region = new_df.loc[idx,'London_Region']
        if not pd.isnull(df.iloc[idx,9]) and df.iloc[idx,9]!=0:
            city = df.iloc[idx,9]
        location = get_location(street, region, city)
        if location != None:
            print(location)
            df.iloc[idx,8] = str(location.latitude) + ", " + str(location.longitude)

In [95]:
def update_city(df):
    df.loc[df['is_from_London'].isin(['y','Y']),['City']] = 'London'

In [96]:
for sheet in sheets:
    print('Processing sheet : ' + sheet)
    fname = sheet+'.xlsx'
    
    if os.path.isfile(fname):
        print('File already exists.')
        continue
    
    df = xl.parse(sheet)
    
    df.columns = new_columns + list(df.columns)[len(new_columns):]
    
    df[['London_Street', 'London_Region', 'is_from_London']].astype(str)
    
    update_city(df)
    update_coordinates(df)
    
    df.to_excel(fname)    

Processing sheet : Emma Lyon 1812
File already exists.
Processing sheet : Bolaffey 1820
File already exists.
Processing sheet : Polack 1830
File already exists.
Processing sheet :  Moss 1839
File already exists.
Processing sheet :  Infant 1841
File already exists.
Processing sheet :  VOJ 1842-45
File already exists.
Processing sheet : Belisario 1856
File already exists.
Processing sheet : Project Details
File already exists.
